In [166]:
import pandas as pd
import numpy as np
import time
from itertools import combinations

In [163]:
def h(values):
    return sum([int(x) + 6 for x in values]) % 100000

def create_pairs(items):
    pairs = []
    items = [int(x) for x in items]
    items = sorted (items)
    for i in range(len(items)):
        for j in range(i+1, len(items)):
            pairs.append((items[i],items[j]))
    return pairs

In [153]:
retail = pd.read_csv('http://fimi.uantwerpen.be/data/retail.dat')
records = retail.values.tolist()
for i in range(len(records)):
    records[i] = records[i][0].split()

In [154]:
percentages = [0.2,0.4,0.6,0.8,1]
thresholds = [0.01, 0.02, 0.05]
samples = []
durations = []

# Loop through percentages and make random samples 
for percentage in percentages:
    samples.append(records[0:int(len(records)*percentage)])

In [178]:
c = 0
# For loop to iterate trhough samples list
for s in samples:
    print(f'COMPUTING {percentages[c]*100}% OF DATASET')
    for x in thresholds:
        items = {}
        hash_table = np.zeros(100000)
        support_threshold = len(s)*x

        start = time.time()
        for i in range(len(s)):
            for j in range(len(s[i])):
                if s[i][j] not in items:
                    items[s[i][j]] = 0 
                items[s[i][j]] += 1
                pairs = combinations(s[i],2)
                for pair in pairs:
                    hash_table[h(pair)] += 1

        C1 = pd.DataFrame({
            'Itemset': items.keys(),
            'Support': items.values()
        })
        L1 = C1.loc[(C1['Support']) >= support_threshold]

        bitmap = np.zeros(100000, dtype=bool)
        for i in range(len(bitmap)):
            bitmap[i] = hash_table[i] >= support_threshold

        arr = []
        for i in L1['Itemset']:
            for j in L1['Itemset']:
                if i == j:
                    pass
                else:
                    for k in s:
                        if i in k and j in k and [i,j,0] not in arr and [j,i,0] not in arr:
                            arr.append([i,j,0])

        for i in range(len(arr)):
                    for j in records:
                        if arr[i][0] in j and arr[i][1] in j and bitmap[h((arr[i][0],arr[i][1]))]:
                            arr[i][2] += 1

        C2 = pd.DataFrame(arr, columns=['Item 1','Item 2','Count'])
        L2 = C2.loc[(C2['Count']) >= support_threshold]
        end = time.time()
        duration = end-start
        durations.append(duration)
        print(f'\tFINISHED RUNNING THRESHOLD {x} \t{duration}s')
    print(f'COMPUTED {percentages[c]*100}% OF DATASET\t{durations[-1]+durations[-2]+durations[-3]}s')    
    c+=1
    print('----------------------------------------------------------------------')
print("Durations: \t",durations)

        

COMPUTING 20.0% OF DATASET
	FINISHED RUNNING THRESHOLD 0.01 	191.54964399337769s
	FINISHED RUNNING THRESHOLD 0.02 	60.42910122871399s
	FINISHED RUNNING THRESHOLD 0.05 	49.389907121658325s
COMPUTED 20.0% OF DATASET	301.36865234375s
----------------------------------------------------------------------
COMPUTING 40.0% OF DATASET
	FINISHED RUNNING THRESHOLD 0.01 	260.7220849990845s
	FINISHED RUNNING THRESHOLD 0.02 	103.05204367637634s
	FINISHED RUNNING THRESHOLD 0.05 	100.64083790779114s
COMPUTED 40.0% OF DATASET	464.41496658325195s
----------------------------------------------------------------------
COMPUTING 60.0% OF DATASET
	FINISHED RUNNING THRESHOLD 0.01 	309.3484182357788s
	FINISHED RUNNING THRESHOLD 0.02 	172.04150915145874s
	FINISHED RUNNING THRESHOLD 0.05 	138.13995385169983s
COMPUTED 60.0% OF DATASET	619.5298812389374s
----------------------------------------------------------------------
COMPUTING 80.0% OF DATASET
	FINISHED RUNNING THRESHOLD 0.01 	440.0869679450989s
	FINISHED